In [ ]:
# Import libraries
import pandas as pd
import numpy as np

In [ ]:
#Read input file
df_input = pd.read_excel('data/Case Study for Business Analyst Financing.xlsx',sheet_name='Data')
columns_new = {'Visit date': 'Visit_date', 'Lead Date':'Lead_Date', 'Sale Date':'Sale_Date', 
               'Business Solution':'Business_Solution', 'Is Visit':'Is_Visit', 'Is Lead':'Is_Lead', 'Is Sale':'Is_Sale'}
df_input.rename(columns=columns_new,inplace=True)


In [ ]:
df_input.info()

In [ ]:
df_input.describe()

In [ ]:
from pandas_profiling import ProfileReport

prof = ProfileReport(df_input)
prof.to_file(output_file='Data_profiling.html')

In [ ]:
# Create table for missing data analysis

def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total Missing', 'Percent of Missing'])
    return missing_data

# Analyse missing data
draw_missing_data_table(df_input)

In [ ]:
# Duplicate row check.

print("Checking for duplicate Rows:")
print("-------------------------------------------------------")
print("Before duplicate row check: (Rows, Columns) = ",  df_input.shape)
df_input.drop_duplicates(inplace=True)
print("After duplicate row check:  (Rows, Columns) = ",  df_input.shape)
print("-------------------------------------------------------")

In [ ]:
# Checking invalid date values.
# Replacing these invalid dates with maximum allowed date for the given month.

df_input['Sale_Date_ck']  = df_input[pd.to_datetime(df_input.Sale_Date, errors='coerce').isnull()].Sale_Date
df_input['Sale_Date_ck'] = df_input['Sale_Date_ck'].isnull()
df_input.head(10)

import datetime
import calendar

for i in range(len(df_input)):
    
    if not df_input.loc[i]['Sale_Date_ck']:
        year, month, day = df_input.loc[i]['Sale_Date'].split('-')
        new_date = datetime.datetime(int(year), int(month), int(calendar.monthrange(int(year), int(month))[1]))
        df_input.at[i,'Sale_Date'] = new_date

df_input.drop(labels="Sale_Date_ck",axis=1,inplace=True)

In [ ]:
# Creating New fields.

df_input['Visit_month'] = df_input.Visit_date.dt.to_period('M')
df_input['Lead_month'] = df_input.Lead_Date.dt.to_period('M')
df_input['Sale_month'] = df_input.Sale_Date.dt.to_period('M')

df_input['Visit_date'] = df_input['Visit_date'].map(datetime.datetime.date) 
df_input['Lead_Date'] = df_input['Lead_Date'].map(datetime.datetime.date) 


df_input['Sale_Date'] = df_input.Sale_Date.dt.date
df_input["Visit_to_Lead"] = (df_input.Lead_Date - df_input.Visit_date).dt.days
df_input["Lead_to_Sale"] = (df_input.Sale_Date - df_input.Lead_Date).dt.days
df_input["Visit_to_Sale"] = (df_input.Sale_Date - df_input.Visit_date).dt.days

In [ ]:
df_input.head(10)

In [ ]:
df_input.sort_values(by=["Visit_date","Lead_Date","Sale_Date"],inplace=True)


In [ ]:
df_input.head(5)

In [ ]:
df_input.drop_duplicates(subset=["Visit_date","Lead_Date","Sale_Date","Business_Solution"])

In [ ]:
# Duplicates rows checking with key values
# Keys - Visit_date, Lead_Date, Sale_Date, Business_Solution.

df_input[df_input.duplicated(subset=["Visit_date","Lead_Date","Sale_Date","Business_Solution"],keep=False)].head(10)

In [ ]:
# Aggregating metrcis values using keys.

aggr = {'Is_Visit':'sum', 'Is_Lead':'sum', 'Is_Sale':'sum', 'Sale_month':'max', 'Visit_to_Lead':'max', 'Lead_to_Sale':'max',
       'Visit_to_Sale':'max', 'Visit_month':'max', 'Lead_month':'max'}
df_input_keyagg = df_input.groupby(["Visit_date","Lead_Date","Sale_Date","Business_Solution"],as_index=False).agg(aggr)

In [ ]:
df_input_keyagg.head(5)

In [ ]:
#df_input_keyagg.groupby(['Sale_month','Business_Solution'], as_index=False).agg({'Is_Sale':'sum'}) 

df_input_mnagg = pd.merge(left=(pd.merge(left=df_input_keyagg.groupby(['Visit_month','Business_Solution'], as_index=False).agg({'Is_Visit':'sum'}), 
             right=df_input_keyagg.groupby(['Sale_month','Business_Solution'], as_index=False).agg({'Is_Sale':'sum'}), 
             how='inner', 
             left_on=['Visit_month','Business_Solution'], 
             right_on=['Sale_month','Business_Solution'])),
             right=df_input_keyagg.groupby(['Lead_month','Business_Solution'], as_index=False).agg({'Is_Lead':'sum'}), 
             how='inner', 
             left_on=['Visit_month','Business_Solution'], 
             right_on=['Lead_month','Business_Solution'])

In [ ]:
df_input_mnagg

In [ ]:
#Reading Financial Business data

df_fin = pd.read_excel('data/car_fin.xlsx')

In [ ]:
df_fin.info()

In [ ]:
df_ocdf =  pd.merge(left=df_input_mnagg, right=df_fin, how='inner', 
         left_on=['Visit_month','Business_Solution'], right_on=['Visit_month','Business_Solution'])
df_ocdf

In [ ]:
# Merging monthly data

In [ ]:
df_mnt_stats = df_input_mnagg[['Visit_month']]
df_mnt_stats.drop_duplicates(inplace=True)
df_mnt_stats = df_mnt_stats.reset_index().drop('index',axis=1)

In [ ]:
df_mnt_stats

In [ ]:
df_ocdf
df_mnt_stats

df_summary =  pd.merge(left=df_ocdf, right=df_mnt_stats, how='left', 
                       left_on=['Visit_month'], right_on=['Visit_month'])


In [ ]:
df_summary.to_excel('output/summary.xlsx',sheet_name='Summary')
df_input_keyagg.to_excel('output/Data.xlsx',sheet_name='Data')